In [1]:
import tensorflow as tf
import numpy as np

In [2]:
Blinks = np.load('./npy/Blinks_F3.npy')
Labels = np.load('./npy/Labels_F3.npy')
BlinksTest = np.load('./npy/BlinksTest_F3.npy')
LabelsTest = np.load('./npy/LabelsTest_F3.npy')
    #deciding the indices of each video based on the fold
    #####################Normalizing the input#############Second phase
BlinksTest[:,:,0]=(BlinksTest[:,:,0]-np.mean(Blinks[:,:,0]))/np.std(Blinks[:,:,0])
Blinks[:,:,0]=(Blinks[:,:,0]-np.mean(Blinks[:,:,0]))/np.std(Blinks[:,:,0])
    #####
    #####
BlinksTest[:,:,1]=(BlinksTest[:,:,1]-np.mean(Blinks[:,:,1]))/np.std(Blinks[:,:,1])
Blinks[:,:,1]=(Blinks[:,:,1]-np.mean(Blinks[:,:,1]))/np.std(Blinks[:,:,1])
    #####
BlinksTest[:,:,2]=(BlinksTest[:,:,2]-np.mean(Blinks[:,:,2]))/np.std(Blinks[:,:,2])
Blinks[:,:,2]=(Blinks[:,:,2]-np.mean(Blinks[:,:,2]))/np.std(Blinks[:,:,2])
    #####
BlinksTest[:,:,3]=(BlinksTest[:,:,3]-np.mean(Blinks[:,:,3]))/np.std(Blinks[:,:,3])
Blinks[:,:,3]=(Blinks[:,:,3]-np.mean(Blinks[:,:,3]))/np.std(Blinks[:,:,3])
    ####
    ####JUST TO DOUBLE CHECK
    ####
    # print(np.mean(Blinks[:,:,0]),np.mean(Blinks[:,:,1]),np.mean(Blinks[:,:,2]),np.mean(Blinks[:,:,3]))
    # print(np.std(Blinks[:,:,0]),np.std(Blinks[:,:,1]),np.std(Blinks[:,:,2]),np.std(Blinks[:,:,3]))
    # print(np.mean(BlinksTest[:,:,0]),np.mean(BlinksTest[:,:,1]),np.mean(BlinksTest[:,:,2]),np.mean(BlinksTest[:,:,3]))
    # print(np.std(BlinksTest[:,:,0]),np.std(BlinksTest[:,:,1]),np.std(BlinksTest[:,:,2]),np.std(BlinksTest[:,:,3]))

In [9]:
def create_model(feature_size):
    #Feature Transformation Layers
    input = tf.keras.Input(shape=(None, None, feature_size))
    reshaped_input_net=tf.reshape(input, [-1, feature_size])
    #   F = ReLU(BW + b)
    F = tf.keras.layers.Dense(activation='relu', units=32)(reshaped_input_net)
    F = tf.reshape(F, [-1,30,32])
    F = tf.keras.layers.Dropout(rate=tf.Variable(1.0))(F)
    
    print("suga")

create_model(feature_size=4)

KeyboardInterrupt: 

In [9]:
tf.compat.v1.disable_eager_execution()
training = tf.compat.v1.placeholder(tf.bool,name='phase_train')